In [17]:
# AUTO-SET PROJECT ROOT (contains 'analysis/') – ONE CELL, NO FILES CREATED
import os
from pathlib import Path

# Start from current directory
current = Path.cwd()
original = current

# Walk up until we find a folder that has 'analysis' as a subfolder
while current != current.parent:  # stop at filesystem root
    if (current / "analysis").is_dir():
        break
    current = current.parent
else:
    # If not found, stay where we are and warn
    print("Warning: 'analysis/' folder not found in any parent. Staying in:", original)
    current = original

# Change to the root (the folder *containing* analysis/)
os.chdir(current)
print(f"Working directory set to: {os.getcwd()}")

# Optional: quick sanity check
if (Path.cwd() / "analysis").is_dir():
    print("Success: 'analysis/' directory is ready!")
else:
    print("Warning: Still no 'analysis/' folder. You may need to create it.")

Working directory set to: c:\Users\HP\Desktop\Projects\navigation\9-daniel-cremers-random-motion-collect
Success: 'analysis/' directory is ready!


In [25]:
# analysis/run_analysis.py
import os
import numpy as np
from analysis import (
    load_parquet, load_embeddings, prepare_data, compute_pca_rgb,
    compute_features, align_arrays, get_map_image_path
)
from analysis.plotting import (
    plot_cluster_lidars,
    plot_cluster_on_map,
    plot_embeddings_rgb,
    plot_oriented_embeddings_rgb,
    plot_correlations,
    comprehensive_analysis
)

# ================== CONFIG ==================
DATA_OUTPUT_DIR = "output/2025-11-14-213949_random_walk_10k"
PARQUET_PATH = f"{DATA_OUTPUT_DIR}/merged_100.parquet"
TRAIN_DIR = "train_2025-11-14-215407"
EMBEDDINGS_PATH = f"{DATA_OUTPUT_DIR}/{TRAIN_DIR}/final_embeddings.npy"
SAVE_DIR = f"{DATA_OUTPUT_DIR}/{TRAIN_DIR}/analysis_images"
os.makedirs(SAVE_DIR, exist_ok=True)

# Analysis parameters
MAX_POINTS = 5000  # Reduce for faster processing, increase for more detail
N_CLUSTERS = 20
RANDOM_STATE = 42

# ================== GET MAP IMAGE ==================
map_image_path = get_map_image_path(DATA_OUTPUT_DIR)
print(f"[INFO] Using map image: {map_image_path}")

# ================== LOAD & PREPARE ==================
df = load_parquet(PARQUET_PATH)
embeddings = load_embeddings(EMBEDDINGS_PATH)

# Prepare data and align arrays
x, y, theta, lidar_data = prepare_data(df, embeddings)
rgb = compute_pca_rgb(embeddings)
dist_to_wall, openness, turn_intensity = compute_features(lidar_data, theta)

# Align all arrays
arrays_to_align = (
    x, y, theta,
    dist_to_wall, openness, turn_intensity,
    embeddings, rgb,
    lidar_data  # Add lidar_data for cluster analysis
)

aligned = align_arrays(*arrays_to_align)
x, y, theta, dist_to_wall, openness, turn_intensity, embeddings, rgb, lidar_data = aligned

print("\n=== ARRAY LENGTHS AFTER ALIGNMENT ===")
for name, arr in [
    ("x", x), ("y", y), ("theta", theta),
    ("dist_to_wall", dist_to_wall), ("openness", openness), ("turn_intensity", turn_intensity),
    ("embeddings", embeddings), ("rgb", rgb), ("lidar_data", lidar_data)
]:
    print(f"{name:>15}: {len(arr)}")

assert len({len(a) for a in aligned}) == 1, "Not all arrays have the same length!"

# ================== INDIVIDUAL PLOTS ==================

print("\n=== GENERATING INDIVIDUAL PLOTS ===")

# 1. Cluster LiDARs with temporal separation
print("1. Generating cluster LiDARs plot...")
plot_cluster_lidars(
    embeddings, lidar_data, x, y, theta,
    n_clusters=8,  # Fewer clusters for clearer visualization
    n_samples_per_cluster=8,
    min_temporal_sep=100,
    save_path=os.path.join(SAVE_DIR, "1_cluster_lidars_temporal.pdf"),
    show_map=True,
    data_dir=DATA_OUTPUT_DIR,
    max_points=MAX_POINTS,
    random_state=RANDOM_STATE
)

# 2. Cluster spatial distribution with arrows
print("2. Generating cluster spatial distribution...")
plot_cluster_on_map(
    embeddings, x, y, theta,
    n_clusters=N_CLUSTERS,
    arrow_scale=2,
    save_path=os.path.join(SAVE_DIR, "2_cluster_spatial_arrows.pdf"),
    data_dir=DATA_OUTPUT_DIR,
    max_points=MAX_POINTS,
    random_state=RANDOM_STATE
)

# 3. Full embeddings RGB
print("3. Generating full embeddings RGB plot...")
plot_embeddings_rgb(
    embeddings, x, y,
    save_path=os.path.join(SAVE_DIR, "3_embeddings_rgb_full.pdf"),
    data_dir=DATA_OUTPUT_DIR,
    max_points=MAX_POINTS,
    random_state=RANDOM_STATE
)

# 4. Multiple oriented embeddings
print("4. Generating oriented embeddings plots...")
orientations = [0, 90, 180, 270]
for orientation in orientations:
    plot_oriented_embeddings_rgb(
        embeddings, x, y, theta,
        target_orientation=orientation,
        tolerance=10,
        save_path=os.path.join(SAVE_DIR, f"4_oriented_{orientation}.pdf"),
        data_dir=DATA_OUTPUT_DIR,
        max_points=MAX_POINTS,
        random_state=RANDOM_STATE
    )

# 5. Correlations with features
print("5. Generating correlation plots...")
features_dict = {
    "Distance to Wall": dist_to_wall,
    "Openness": openness,
    "Turn Intensity": turn_intensity
}

# PCA-RGB correlations
plot_correlations(
    features_dict,
    rgb=rgb,
    use_pca=True,
    save_path=os.path.join(SAVE_DIR, "5_correlations_pca_rgb.pdf"),
    corr_type='pearson'
)

# Full embeddings correlations (optional - can be slow for high dimensions)
if embeddings.shape[1] <= 64:  # Only if embeddings are not too high-dimensional
    plot_correlations(
        features_dict,
        embeddings=embeddings,
        use_pca=False,
        save_path=os.path.join(SAVE_DIR, "5_correlations_full.pdf"),
        corr_type='pearson'
    )

# ================== COMPREHENSIVE ANALYSIS ==================

print("\n=== RUNNING COMPREHENSIVE ANALYSIS ===")
comprehensive_analysis(
    embeddings, x, y, theta, lidar_data,
    data_dir=DATA_OUTPUT_DIR,
    save_dir=os.path.join(SAVE_DIR, "comprehensive"),
    max_points=MAX_POINTS,
    n_clusters=N_CLUSTERS,
    random_state=RANDOM_STATE
)

# ================== ADDITIONAL SPECIALIZED PLOTS ==================

print("\n=== GENERATING SPECIALIZED PLOTS ===")

# 6. Multiple cluster spatial distributions
print("6. Generating multiple cluster views...")
for i in range(3):  # Generate 3 different random clusters
    plot_cluster_on_map(
        embeddings, x, y, theta,
        n_clusters=N_CLUSTERS,
        arrow_scale=2,
        save_path=os.path.join(SAVE_DIR, f"6_cluster_example_{i+1}.pdf"),
        data_dir=DATA_OUTPUT_DIR,
        max_points=MAX_POINTS,
        random_state=RANDOM_STATE + i  # Different seed for each
    )

# 7. High temporal separation analysis
print("7. Generating high temporal separation analysis...")
plot_cluster_lidars(
    embeddings, lidar_data, x, y, theta,
    n_clusters=6,
    n_samples_per_cluster=6,
    min_temporal_sep=500,  # Higher separation to show generalization
    save_path=os.path.join(SAVE_DIR, "7_cluster_lidars_high_sep.pdf"),
    show_map=True,
    data_dir=DATA_OUTPUT_DIR,
    max_points=MAX_POINTS,
    random_state=RANDOM_STATE
)

# 8. Different arrow scales for orientation visualization
print("8. Generating orientation visualization with different scales...")
for arrow_scale in [1, 2, 4]:
    plot_cluster_on_map(
        embeddings, x, y, theta,
        n_clusters=min(N_CLUSTERS, 10),
        arrow_scale=arrow_scale,
        save_path=os.path.join(SAVE_DIR, f"8_orientation_scale_{arrow_scale}.pdf"),
        data_dir=DATA_OUTPUT_DIR,
        max_points=min(MAX_POINTS, 2000),  # Fewer points for clarity
        random_state=RANDOM_STATE
    )

print(f"\n=== ANALYSIS COMPLETE ===")
print(f"All results saved to: {SAVE_DIR}")
print(f"Total points used: {len(x)}")
print(f"Maximum points parameter: {MAX_POINTS}")
print(f"Number of clusters: {N_CLUSTERS}")

# Summary of generated files
print("\n=== GENERATED FILES ===")
generated_files = [
    "1_cluster_lidars_temporal.pdf - LiDAR clusters with temporal separation",
    "2_cluster_spatial_arrows.pdf - Spatial cluster distribution with orientation arrows",
    "3_embeddings_rgb_full.pdf - Full embeddings colored by PCA-RGB",
    "4_oriented_*.pdf - Orientation-filtered embeddings (0°, 90°, 180°, 270°)",
    "5_correlations_*.pdf - Feature correlation analysis",
    "6_cluster_example_*.pdf - Multiple random cluster examples",
    "7_cluster_lidars_high_sep.pdf - High temporal separation clusters",
    "8_orientation_scale_*.pdf - Orientation visualization with different arrow scales",
    "comprehensive/ - Full analysis pipeline results"
]

for file_desc in generated_files:
    print(f"  • {file_desc}")

[INFO] Using map image: environments\images\6.png
[INFO] Loading output/2025-11-14-213949_random_walk_10k/merged_100.parquet ...
[INFO] Total columns: 105
[INFO] Loaded embeddings: (9998, 64)

=== ARRAY LENGTHS AFTER ALIGNMENT ===
              x: 9998
              y: 9998
          theta: 9998
   dist_to_wall: 9998
       openness: 9998
 turn_intensity: 9998
     embeddings: 9998
            rgb: 9998
     lidar_data: 9998

=== GENERATING INDIVIDUAL PLOTS ===
1. Generating cluster LiDARs plot...
[INFO] Subsampling to 5000 points...
[INFO] Saved cluster LiDARs to output/2025-11-14-213949_random_walk_10k/train_2025-11-14-215407/analysis_images\1_cluster_lidars_temporal.pdf
2. Generating cluster spatial distribution...
[INFO] Subsampling to 5000 points...
[INFO] Saved cluster map to output/2025-11-14-213949_random_walk_10k/train_2025-11-14-215407/analysis_images\2_cluster_spatial_arrows.pdf
3. Generating full embeddings RGB plot...
[INFO] Subsampling to 5000 points...
[INFO] Saved RGB e